In [1]:
import openai
import time
import random 
from tqdm import tqdm

# Set your API key
openai.api_key = # add ur code

In [2]:
# # this cell test if api works



# response = openai.ChatCompletion.create(
#     model="gpt-4o",
#     messages=[
#         {"role": "user", "content": "what is one plus one."}
#     ]
# )

# # Print the assistant's response
# print(response['choices'][0]['message']['content'])

In [3]:
import requests
import json
import pandas as pd

# Box direct link to json
urls = {
    "en_train": "https://uwmadison.box.com/shared/static/6vpi1ee13l0lx6jbh9dtmf9nvfdlhqtb.jsonl",
    "en_valid": "https://uwmadison.box.com/shared/static/pfp3yhkl2xw4f4l4ujrxjek6j34y3bp4.jsonl",
    "en_test":  "https://uwmadison.box.com/shared/static/wsqy6m9jx278ke5dm72kbn7o3yb2zmph.jsonl",
    "ch_train": "https://uwmadison.box.com/shared/static/waemr4sstq01xv510k5423jck7maubd9.jsonl",
    "ch_valid": "https://uwmadison.box.com/shared/static/fnf0ug42jm3fbqu41uhbwuq0oobhak6r.jsonl",
    "ch_test":  "https://uwmadison.box.com/shared/static/ra4m08ackxvsz62aknesjes6oyxlwn6j.jsonl"
}

# Download and parse the data
dataset = {}
for name, url in urls.items():
    response = requests.get(url)
    lines = response.text.strip().split('\n')
    dataset[name] = [json.loads(line) for line in lines]


In [6]:
toy_dataset = {
    
    'en_train': 
    [{'content': "In February, Alicia Silverstone and Christopher Jarecki announced they were divorcing after 20 years together. The couple has a six - year - old son together, Bear Blu. ( Photo : Frazer Harrison, Getty Images ) Alicia Silverstone has filed for divorce from her husband of nearly 13 years, actor and musician Christopher Jarecki. The divorce papers were filed in Los Angeles County Superior Court on Friday, according to the Associated Press. The Clueless star, 41, had separated from Jarecki, 47, in February after more than 20 years together as a couple. At the time the couple said in a statement that   they still deeply love and respect each other and remain very close friends.'' The papers state the couple will share custody of their 7 - year - old son, Bear Blue",
   'label': 0,
   'split': 'test'},
     
  {'content': 'Does Meghan Markle Want 5 Kids? Prince Harry Has Some Thoughts The royal newlyweds may still be basking in their honeymoon glow, but of course people are already wondering if Prince Harry and Meghan Markle want kids. When pressed by fans, the Prince has made it clear exactly how many kids is too many, but that doesn ’ t mean babies won ’ t be part of the couple ’ s future. Prince Harry and Meghan Markle have already been forthcoming about wanting to have a family. During their first interview with the BBC after getting engaged, they were questioned about having kids and Prince Harry responded, ” I think one step at a time and hopefully we ’ ll start a family in the near future. ” ADVERTISEMENT The question is, exactly how many kids? With big brother',
   'label': 0,
   'split': 'test'}],
    
    'ch_train': [{'content': '全国首家无人银行亮相大堂经理是智能机器人银行里没有忙碌的工作人员和拥挤的排队客户，而是各种机器人、智慧柜员机、存取款机、外汇兑换机等智能化设备，科技感十足。业内观点无人银行是银行物理网点的一个创新，是转型的一种选择，但并不意味着会成为一种趋势，线上银行才是未来银行业发展开全文c',
   'label': 0,
   'split': 'test'},
  {'content': '【媒体融合链来了中国搜索赋能版权保护】媒体融合链，版权价值链。由新华社中国搜索研发的媒体融合链区块链版权平台，近日正式上线。作为国家版权局版权保护新技术研究推广站点授牌单位，中国搜索此举旨在打造融合、协同、创新的版权新生态，助力提升媒体版权价值。详戳：...全文',
   'label': 0,
   'split': 'test'}]
     }


In [5]:
# prompt

textual_prompt_template = """
    "You are a truthful news analyzer. Analyze the following news report purely based on its textual content and writing style. "
    "List key points about the writing (level of detail, sources cited, quotes, consistency, tone, etc.) that indicate whether the news is likely true or false. "
    "After listing these points, conclude with a sentence stating your judgment ('Based on these textual characteristics, the news is likely real' or '... likely fake')."
    "limite the response to 150 words. "
    "News: {content}"
    "Your analysis:"
"""

commonsense_prompt_template = """
    "You are a fact-checking expert with common sense. Analyze the veracity of the following news report based on real-world logic and verifiability. "
    "Consider the following aspects:"
    "1. Plausibility: Is the scenario logical and plausible?"
    "2. Verifiability: Can the claims be verified by known facts or public records?"
    "3. Source credibility: Is the source (if any mentioned) trustworthy?"
    "Provide analysis for each aspect, then conclude with a sentence stating your judgment ('Based on these factors, the news is likely real' or '... likely fake')."
    "limite the response to 150 words. "
    "News: {content}"
    "Your analysis:"
"""



# English

## Expert View: 
start time 7:07, starting tokens 5284k; 
end time 10:47, end tokens 6369k

- 3h40min, used 1086k tokens

In [12]:
# add expert view first

en_train = []
rationales = []

for item in dataset['en_train']:
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": textual_prompt_template },
                    {"role": "user", "content": item['content']},
                ],
                temperature=0.1,
                max_tokens = 200
            )
            break  # break the loop if no exception
        except openai.error.RateLimitError:
            print("Rate limit hit, sleeping for 1 second...")
            time.sleep(1)  # wait and retry

    rationale = response['choices'][0]['message']['content'].strip()
    rationales.append(rationale)

    label = item['label']

    en_train.append({
        "content": item['content'],
        "label": label,
        "textual": rationale
    })


In [13]:
# break the pervious loop to see how long should I wait! 
# then I run the rest of them

# Save to JSON


with open("rational_expert_en_train2.json", "w", encoding="utf-8") as f:
    json.dump({"en_train": en_train}, f, indent=2, ensure_ascii=False)

In [23]:
with open("rational_expert_en_train.json", "r", encoding="utf-8") as f:
    rational_expert_en_train = json.load(f)

rational_expert_en_train


In [40]:
# # then add common sense

# en_train = []
# rationales = []

# for item in rational_expert_en_train['en_train']:
#     while True:
#         try:
#             response = openai.ChatCompletion.create(
#                 model="gpt-4o",
#                 messages=[
#                     {"role": "system", "content": commonsense_prompt_template },
#                     {"role": "user", "content": item['content']},
#                 ],
#                 temperature=0.1,
#                 max_tokens = 200
#             )
#             break  # break the loop if no exception
#         except openai.error.RateLimitError:
#             print("Rate limit hit, sleeping for 1 second...")
#             time.sleep(1)  # wait and retry

#     rationale = response['choices'][0]['message']['content'].strip()
#     rationales.append(rationale)

#     en_train.append({
#         "content": item['content'],
#         "label": item['label'],
#         "textual": item['textual'],
#         "common_sense": rationale
#     })

# # Save to JSON
# with open("rationale_en_train.json", "w", encoding="utf-8") as f:
#     json.dump({"en_train": en_train}, f, indent=2, ensure_ascii=False)


In [41]:
# with open("rationale_en_train.json", "r", encoding="utf-8") as f:
#     rationale_en_train = json.load(f)

# rationale_en_train

{'en_train': [{'content': "In February, Alicia Silverstone and Christopher Jarecki announced they were divorcing after 20 years together. The couple has a six - year - old son together, Bear Blu. ( Photo : Frazer Harrison, Getty Images ) Alicia Silverstone has filed for divorce from her husband of nearly 13 years, actor and musician Christopher Jarecki. The divorce papers were filed in Los Angeles County Superior Court on Friday, according to the Associated Press. The Clueless star, 41, had separated from Jarecki, 47, in February after more than 20 years together as a couple. At the time the couple said in a statement that   they still deeply love and respect each other and remain very close friends.'' The papers state the couple will share custody of their 7 - year - old son, Bear Blue",
   'label': 0,
   'textual': "Your analysis:\n\n1. **Level of Detail**: The report provides specific details such as the names of the individuals involved, their ages, the duration of their relationsh

# Chinese 

start 6372 tokens, end 7490 tokens, 

used 1118 tokens, total run time 4h 43 min

In [6]:
# add expert view first

ch_train = []
rationales = []

for item in tqdm(dataset['ch_train'], desc="I'm a hard working program"):
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": textual_prompt_template},
                    {"role": "user", "content": item['content']},
                ],
                temperature=0.1,
                max_tokens=200
            )
            break
        except openai.error.RateLimitError:
            print("Rate limit hit, sleeping for 1 second...")
            time.sleep(1)

    rationale = response['choices'][0]['message']['content'].strip()
    rationales.append(rationale)

    label = item['label']

    ch_train.append({
        "content": item['content'],
        "label": label,
        "textual": rationale
    })


I'm a hard working program: 100%|████████████████████████████████████████████████| 5204/5204 [4:43:03<00:00,  3.26s/it]


In [7]:
with open("rational_expert_ch_train.json", "w", encoding="utf-8") as f:
    json.dump({"ch_train": ch_train}, f, indent=2, ensure_ascii=False)

with open("rational_expert_ch_train.json", "r", encoding="utf-8") as f:
    rational_expert_ch_train = json.load(f)


{'ch_train': [{'content': '【能电鱼，更能电力找鱼：别以为电鳗只有把鱼电晕一招！】电鳗2米长的身体里有许多专门发电的细胞，能产生600伏电击，足以电倒一个人。以为电鳗就这么点本事？最新研究发现：它还能把猎物从藏身之处电出来，甚至能靠放电追踪猎物。http://t.cn/runzfkg但像超凡蜘蛛侠那样电出超能力是不行的[微笑]',
   'label': 0,
   'textual': 'Your analysis:\n\n1. **Level of Detail**: The report provides specific details about the electric eel, such as its length and the voltage it can produce, which adds credibility.\n2. **Sources Cited**: There is a link provided (http://t.cn/runzfkg), suggesting that there might be a source or further reading, although the link itself is not verifiable in this context.\n3. **Quotes**: The article does not include direct quotes from experts or studies, which could have strengthened its credibility.\n4. **Consistency**: The information about electric eels aligns with known scientific facts, such as their ability to generate electricity.\n5. **Tone**: The tone is informative with a hint of humor, especially in the reference to "superpowers," which might be intended to engage readers.\n